# TIPS

CONVERT(Slot_begin, integer)

( assets.`CONTAINER` LIKE '%uest' OR
   assets.`CONTAINER` = 'SUN zone' OR
   assets.`CONTAINER` = 'HP vpar' )

# Creation / Alteration de table

In [ ]:
CREATE VIEW `assets_FRVLB0C2_FRTCRAVA`
as
select *
from`assets_new`
where (API = 'FRVLB0C2' or API = 'FRTCRAVA')

In [ ]:
CREATE TABLE assets_filtree
as
select *
from `assets_new` 
WHERE 
    ETAT = 'Installed' 
    and (API = 'FRVLB0C2' or API = 'FRTCRAVA')
    and ( assets.`CONTAINER` LIKE '%uest' OR -- Distinction entre machine virtuelle et machine physique 
   assets.`CONTAINER` = 'SUN zone' OR
   assets.`CONTAINER` = 'HP vpar' )

In [ ]:
table avec les matchs assets ( 1867 ) 

CREATE TABLE assets_match 
as
select *
FROM assets_filtree
where STATUS is not null

In [ ]:
create table IT_Equipment_match
as
select *
from `IT_Equipment`
where STATUS is not null

In [ ]:
Ajout des colonnes pour les coordonnees d'equipements
ALTER TABLE assets_filtree
ADD COLUMN Rack varchar(255),
ADD COLUMN Slot varchar(255),
ADD COLUMN Slot_begin varchar(255),
ADD COLUMN Slot_end varchar(255);

# Compteur

SELECT count(*) count, TYPE
from `IT_Equipment`
where STATUS is null
GROUP BY TYPE

In [ ]:
SELECT count(*) count, YPE
from `assets_filtree`
where STATUS is null
GROUP BY TYPE

select COUNT(STATUS), status
from `assets_filtree`
GROUP BY STATUS
ORDER BY COUNT(STATUS) desc

select COUNT(STATUS), status
from `IT_Equipment`
GROUP BY STATUS
ORDER BY COUNT(STATUS) desc

select count(*)
from `IT_Equipment`
where status is null

# Query

Check si y a des noms courts qui correspondent entre eux

SELECT * 
from `assets_filtree` aa
INNER JOIN `IT_Equipment` it ON aa.nom_court = it.short_name

In [ ]:
Asset_id équivalent mais nom différents ( on y rajoute un status different)

select aa.asset_id, it.asset_id asset_id_id, aa.nom_court, it.short_name
from `assets_filtree` aa
INNER JOIN `IT_Equipment` it on aa.asset_id = it.asset_id
where STATUS is null

### SERVER TYPE

Retire les 3 derniers caractères des nom de serveurs (43 matchs)
select aa.asset_id, it.asset_id asset_id_id, aa.nom_court, it.short_name
from `assets_filtree` aa
INNER JOIN `IT_Equipment` it on LEFT(aa.nom_court, length(aa.nom_court) - 3) = it.short_name
where it.type = 'Serveurs'

In [ ]:
select aa.asset_id, it.asset_id asset_id_id, aa.nom_court, it.short_name, aa.status, it.status status_it
from `IT_Equipment` it
INNER JOIN `assets_filtree` aa on LEFT(aa.nom_court, length(aa.nom_court) - 3) = it.short_name

### Match sur les coordonnees

In [ ]:
SELECT it.Nom_de_la_salle, it.Nom_de_baie, it.Slot_U, it.Reperage_ID, it.Hauteur_U 
FROM `IT_Equipment` it
where it.Nom_de_baie = 'S08'

DALLE, Rack , Slot, Slot_begin, Slot_end, Salle

SELECT DALLE,
	   SUBSTRING_INDEX(DALLE, '/', 1) Rack ,
       SUBSTRING_INDEX(DALLE, '/', -1) Slot,
       SUBSTRING_INDEX(SUBSTRING_INDEX(DALLE, '/', -1), '-', 1) Slot_begin,
       SUBSTRING_INDEX(DALLE, '-', -1) Slot_end

FROM `assets_filtree`

Query des positions en salle pour assets
SELECT Rack,
Slot,
Slot_begin,
Slot_end,
SALLE
from `assets_filtree`

(453 total, Query took 9.1079 seconds.)
SELECT *
FROM `assets_filtree` aa
INNER JOIN 
	`IT_Equipment` it 
ON 
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = aa.Slot_Begin
AND
	it.Hauteur_U = aa.Slot_End

In [ ]:
41 rows affected. (Query took 51.4745 seconds.)
SELECT aa.YPE, aa.SUBTYPE, aa.ASSET_ID, it.asset_id asset_id_it, aa.NOM_COURT, it.short_name, aa.NOM_LONG, it.nom, aa.CRITICITE, aa.FABRICANT, it.fabricant fabricant_it, aa.MODEL, it.modele modele_it,
aa.Salle, it.Nom_de_la_salle, aa.Rack, it.Nom_de_baie, aa.Slot_begin, it.Slot_U, aa.Slot_end , it.Hauteur_U

FROM `assets_filtree` aa
INNER JOIN 
	`IT_Equipment` it 
ON 
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
AND
	(it.Hauteur_U = CONVERT(aa.Slot_End, integer) or it.Hauteur_U = CONVERT(aa.Slot_End, integer) - CONVERT(aa.Slot_Begin, integer))

WHERE aa.status = 'OK-POSITION' and it.status = 'OK-POSITION'

### Match sur les differences entres nom long et nom court

select aa.asset_id, it.asset_id asset_id_id, aa.nom_court, it.short_name, aa.nom_long, it.nom, aa.status, it.status status_it
from `assets_filtree` aa
INNER JOIN `IT_Equipment` it on 
Upper(aa.nom_court)= upper(it.short_name) or Upper(aa.nom_long)= upper(it.nom) or
 Upper(aa.nom_court)= upper(it.nom) or Upper(aa.nom_long)= upper(it.short_name)
where aa.status is null

### Doublon physique/Virtuel

select * from `assets_filtree`
where ASSET_ID = 'M-0000220405' or ASSET_ID = 'M-0000221162'

select * from `IT_Equipment` where ASSET_ID = 'M-0000324239' 

# Update

1818 rows affected. (Query took 94.4469 seconds.) - 

UPDATE `assets_filtree` aa
INNER JOIN `IT_Equipment` it ON aa.nom_court = it.short_name
SET aa.status = 'OK-CMDB-ASSET-ID'

10 rows affected. (Query took 94.6242 seconds.)
UPDATE `assets_filtree` aa
INNER JOIN `IT_Equipment` it on aa.asset_id = it.asset_id
set aa.status = 'ASSET_ID OK / SHORT_NAME MISMATCH' 
where aa.STATUS is null

In [ ]:
39 rows affected. (Query took 65.3069 seconds.) -  
UPDATE `assets_filtree` aa
INNER JOIN `IT_Equipment` it on LEFT(aa.nom_court, length(aa.nom_court) - 3) = it.short_name
SET aa.status = it.asset_id -- on remplace par l'asset id de la table Equipements
where it.type = 'Serveurs' and STATUS IS NULL

In [ ]:
UPDATE `IT_Equipment` it
INNER JOIN `assets_filtree` aa ON aa.asset_id = it.asset_id
SET it.status = 'OK-CMDB-ASSET-ID

UPDATE `IT_Equipment` it
INNER JOIN `assets_filtree` aa on aa.asset_id = it.asset_id
set it.status = 'ASSET_ID OK / SHORT_NAME MISMATCH' 
where STATUS is null

UPDATE `IT_Equipment` it
INNER JOIN `assets_filtree` aa on LEFT(aa.nom_court, length(aa.nom_court) - 3) = it.short_name
SET it.status = aa.asset_id -- on remplace par l'asset id de la table assets
where it.type = 'Serveurs' and it.STATUS IS NULL

In [ ]:
Distinction entre les equipements physique ou non

UPDATE `assets_filtree` assets
SET assets.emplacement = 'Physical'
WHERE ( assets.`API` = 'FRVLB0C2' OR 
    assets.`API` = 'FRTCRGRA' OR
    assets.`API` = 'FRTCRAVA' OR
    assets.`API` = 'FREQXPA4' OR
    assets.`API` = 'FREQXPA6' ) 
    AND ( assets.`CONTAINER` LIKE '%host' OR
    assets.`CONTAINER` = 'Appliance' OR
    assets.`CONTAINER` = 'OVM vm host' OR
    assets.`CONTAINER` = 'HP npar' OR
    assets.`CONTAINER` = 'IBM lpar' OR
    assets.`CONTAINER` = 'IBM non-X86' OR
    assets.`CONTAINER` = 'Part. logique' OR
    assets.`CONTAINER` = 'Physique' OR
    assets.`CONTAINER` = 'Standalone' OR
    assets.`CONTAINER` = 'standalone' OR
    assets.`CONTAINER` = 'SUN domain' OR
    assets.`CONTAINER` = 'SUN global zone' )
    AND assets.`ETAT` = 'Installed'
    AND ( assets.`SUBTYPE` = 'Server' OR
    assets.`SUBTYPE` = 'Appliance' )

Ajout des colonnes des coordonnees en salle
UPDATE assets_filtree
SET 
	Rack = SUBSTRING_INDEX(DALLE, '/', 1)  ,
	Slot = SUBSTRING_INDEX(DALLE, '/', -1) ,
	Slot_begin = SUBSTRING_INDEX(SUBSTRING_INDEX(DALLE, '/', -1), '-', 1) ,
	Slot_end = SUBSTRING_INDEX(DALLE, '-', -1)

In [ ]:
UPDATE `assets_filtree` aa
INNER JOIN 
	`IT_Equipment` it 
ON 
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
AND(
	it.Hauteur_U = CONVERT(aa.Slot_End, integer) or it.Hauteur_U = CONVERT(aa.Slot_End, integer) - CONVERT(aa.Slot_Begin, integer))

SET aa.status = 'OK-POSITION',
	it.status = 'OK-POSITION'

WHERE aa.status is null and it.status is null

198 rows affected. (Query took 113.2333 seconds.)
-- Match sur les differences entres nom long et nom court

Update `assets_filtree` aa
INNER JOIN `IT_Equipment` it on 
Upper(aa.nom_court)= upper(it.short_name) or Upper(aa.nom_long)= upper(it.nom) or
Upper(aa.nom_court)= upper(it.nom) or Upper(aa.nom_long)= upper(it.short_name)
SET aa.status = 'Match-nom_long-nom_court',
	it.status = 'Match-nom_long-nom_court'
where aa.status is null and it.status is null

10 rows affected. (Query took 19.5651 seconds.)
Update `IT_Equipment` it 
INNER JOIN  on `assets_filtree` aa
Upper(aa.nom_court)= upper(it.short_name) or Upper(aa.nom_long)= upper(it.nom) or
Upper(aa.nom_court)= upper(it.nom) or Upper(aa.nom_long)= upper(it.short_name)
SET it.status = 'Match-nom_long-nom_court'
where it.status is null